In [1]:
import numpy as np
import pandas as pd

# 导入数据

In [2]:
df = pd.read_csv(rf'.\data\数据1.csv', dtype={'日期':'str', '代理商id':str})

df.dtypes

df = df[df.代理商id.isin(['1505810869882216449', '1232928398737735682', '1237976749447380993'])]

In [3]:
df

,日期,代理商id,当日销售额
0,2021-01-01,1232928398737735682,177282.0
2,2021-01-01,1237976749447380993,96340.0
41,2021-01-01,1505810869882216449,388684.0
46,2021-01-02,1232928398737735682,140710.0
48,2021-01-02,1237976749447380993,101642.0
...,...,...,...
54834,2024-04-07,1237976749447380993,21086.0
54873,2024-04-07,1505810869882216449,87958.0
54878,2024-04-08,1232928398737735682,15790.0
54880,2024-04-08,1237976749447380993,20486.0


# 创建完整维度表

In [4]:
df.日期.unique()

array(['2021-01-01', '2021-01-02', '2021-01-03', ..., '2024-04-06',
       '2024-04-07', '2024-04-08'], dtype=object)

In [5]:
df.代理商id.unique()

array(['1232928398737735682', '1237976749447380993',
       '1505810869882216449'], dtype=object)

In [6]:
df.日期.min()

'2021-01-01'

In [7]:
df.日期.max()

'2024-04-08'

In [8]:
# 完整日期
date_list = pd.date_range(df.日期.min(), df.日期.max()).map(lambda x:str(x)[0:10])
date_list

Index(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05',
       '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10',
       ...
       '2024-03-30', '2024-03-31', '2024-04-01', '2024-04-02', '2024-04-03',
       '2024-04-04', '2024-04-05', '2024-04-06', '2024-04-07', '2024-04-08'],
      dtype='object', length=1194)

In [9]:
len(date_list)

1194

In [10]:
# 完整的经销商
agent_id_list = df.代理商id.unique()
agent_id_list

array(['1232928398737735682', '1237976749447380993',
       '1505810869882216449'], dtype=object)

In [11]:
pd.DataFrame(date_list)

,0
0,2021-01-01
1,2021-01-02
2,2021-01-03
3,2021-01-04
4,2021-01-05
...,...
1189,2024-04-04
1190,2024-04-05
1191,2024-04-06
1192,2024-04-07


In [12]:
L = []
for i in agent_id_list:
    dfi = pd.DataFrame(date_list)
    dfi[1] = i
    L.append(dfi)

In [13]:
dd = pd.concat(L)
dd

,0,1
0,2021-01-01,1232928398737735682
1,2021-01-02,1232928398737735682
2,2021-01-03,1232928398737735682
3,2021-01-04,1232928398737735682
4,2021-01-05,1232928398737735682
...,...,...
1189,2024-04-04,1505810869882216449
1190,2024-04-05,1505810869882216449
1191,2024-04-06,1505810869882216449
1192,2024-04-07,1505810869882216449


In [14]:
dd.columns = ['日期', '代理商id']

In [15]:
dd

,日期,代理商id
0,2021-01-01,1232928398737735682
1,2021-01-02,1232928398737735682
2,2021-01-03,1232928398737735682
3,2021-01-04,1232928398737735682
4,2021-01-05,1232928398737735682
...,...,...
1189,2024-04-04,1505810869882216449
1190,2024-04-05,1505810869882216449
1191,2024-04-06,1505810869882216449
1192,2024-04-07,1505810869882216449


In [16]:
dd.代理商id.value_counts()

1232928398737735682    1194
1237976749447380993    1194
1505810869882216449    1194
Name: 代理商id, dtype: int64

# 合并完整维度表

In [17]:
dd = dd.merge(df, how='left', on=['日期', '代理商id']).fillna(0)

In [18]:
dd

,日期,代理商id,当日销售额
0,2021-01-01,1232928398737735682,177282.0
1,2021-01-02,1232928398737735682,140710.0
2,2021-01-03,1232928398737735682,120226.0
3,2021-01-04,1232928398737735682,28482.0
4,2021-01-05,1232928398737735682,49168.0
...,...,...,...
3577,2024-04-04,1505810869882216449,130838.0
3578,2024-04-05,1505810869882216449,203100.0
3579,2024-04-06,1505810869882216449,295154.0
3580,2024-04-07,1505810869882216449,87958.0


# 固定统计周期的计算

## 过去n天的聚合
- Series.rolling(window=N,min_periods=0).sum()
- Series.rolling(window=N,min_periods=0).mean()等等
- 这是统计前面N行（包括当前行）数据的聚合

In [19]:
x = pd.Series([1, 2, np.nan, np.nan, 3, 4])
x

0    1.0
1    2.0
2    NaN
3    NaN
4    3.0
5    4.0
dtype: float64

In [20]:
x.rolling(window=2,min_periods=0).sum() # 过去2条求和

0    1.0
1    3.0
2    2.0
3    0.0
4    3.0
5    7.0
dtype: float64

In [21]:
def func(dfi):
    # 要先排序
    dfi = dfi.sort_values('日期', ascending=True)
    
    # 然后计算
    dfi['近7日销售额'] = dfi['当日销售额'].rolling(window=7,min_periods=0).sum()
    dfi['近14日销售额'] = dfi['当日销售额'].rolling(window=14,min_periods=0).sum()
    dfi['近30日销售额'] = dfi['当日销售额'].rolling(window=30,min_periods=0).sum()
    dfi['近60日销售额'] = dfi['当日销售额'].rolling(window=60,min_periods=0).sum()
    
    dfi['近7日日均销售额'] = dfi['当日销售额'].rolling(window=7,min_periods=0).mean()
    return dfi

dd.groupby('代理商id', as_index=False).apply(func)

,日期,代理商id,当日销售额,近7日销售额,近14日销售额,近30日销售额,近60日销售额,近7日日均销售额
0,2021-01-01,1232928398737735682,177282.0,177282.0,177282.0,177282.0,177282.0,177282.000000
1,2021-01-02,1232928398737735682,140710.0,317992.0,317992.0,317992.0,317992.0,158996.000000
2,2021-01-03,1232928398737735682,120226.0,438218.0,438218.0,438218.0,438218.0,146072.666667
3,2021-01-04,1232928398737735682,28482.0,466700.0,466700.0,466700.0,466700.0,116675.000000
4,2021-01-05,1232928398737735682,49168.0,515868.0,515868.0,515868.0,515868.0,103173.600000
...,...,...,...,...,...,...,...,...
3577,2024-04-04,1505810869882216449,130838.0,789906.0,1570716.0,3643334.0,9469206.0,112843.714286
3578,2024-04-05,1505810869882216449,203100.0,911750.0,1690860.0,3762578.0,9499998.0,130250.000000
3579,2024-04-06,1505810869882216449,295154.0,1036286.0,1836190.0,3967984.0,9602452.0,148040.857143
3580,2024-04-07,1505810869882216449,87958.0,892762.0,1733648.0,3544802.0,9456634.0,127537.428571


In [22]:
## 验证
dd[(dd.代理商id=='1505810869882216449')&(dd.日期<='2024-04-07')].tail(14)['当日销售额'].sum()

1733648.0

In [23]:
## 验证
dd[(dd.代理商id=='1505810869882216449')&(dd.日期<='2024-04-07')].tail(60)['当日销售额'].sum()

9456634.0

In [24]:
## 验证
dd[(dd.代理商id=='1505810869882216449')&(dd.日期<='2024-04-07')].tail(7)['当日销售额'].mean()

127537.42857142857

## 固定时间的聚合
- 当周、当月、当季、当年、历史

In [33]:
a = dd[(dd.代理商id=='1505810869882216449')]
a

,日期,代理商id,当日销售额
2388,2021-01-01,1505810869882216449,388684.0
2389,2021-01-02,1505810869882216449,235560.0
2390,2021-01-03,1505810869882216449,338014.0
2391,2021-01-04,1505810869882216449,203288.0
2392,2021-01-05,1505810869882216449,134530.0
...,...,...,...
3577,2024-04-04,1505810869882216449,130838.0
3578,2024-04-05,1505810869882216449,203100.0
3579,2024-04-06,1505810869882216449,295154.0
3580,2024-04-07,1505810869882216449,87958.0


### 当周第一天

In [97]:
# 这天是星期二
x = pd.to_datetime('2024-04-23')
x

Timestamp('2024-04-23 00:00:00')

In [98]:
# 返回该日期是当前周的周几，周一是0， 周二1，一直到6
x.weekday()

1

In [99]:
# 获取该日期所在周的周一
first_day_of_week = x - pd.DateOffset(days=x.weekday()) 
first_day_of_week

Timestamp('2024-04-22 00:00:00')

In [100]:
def first_day_of_week_func(date):
    x = pd.to_datetime(date)
    first_day_of_week = x - pd.DateOffset(days=x.weekday()) 
    return str(first_day_of_week)[0:10]

In [104]:
first_day_of_week_func('2024-04-23')

'2024-04-22'

### 当月第一天

In [106]:
def first_day_of_month_func(date):
    first_day_of_month = str(date)[0:7]+'-01'
    return first_day_of_month

In [108]:
first_day_of_month_func('2024-04-23')

'2024-04-01'

### 当季第一天

In [132]:
date = pd.to_datetime('2024-02-23')
date

Timestamp('2024-02-23 00:00:00')

In [133]:
# 返回日期所在的月份
date.month

2

In [134]:
# 确定季度的起始月份：1（Q1），4（Q2），7（Q3），10（Q4）  
quarter_start_month = (date.month - 1) // 3 * 3 + 1 
quarter_start_month

1

In [136]:
# 设置日期为该季度的第一天  
first_day_of_quarter = pd.Timestamp(year=date.year, month=quarter_start_month, day=1) 
first_day_of_quarter

Timestamp('2024-01-01 00:00:00')

In [153]:
def first_day_of_quarter_func(date):
    date = pd.to_datetime(date)
    # 确定季度的起始月份：1（Q1），4（Q2），7（Q3），10（Q4）  
    quarter_start_month = (date.month - 1) // 3 * 3 + 1
    # 设置日期为该季度的第一天  
    first_day_of_quarter = pd.Timestamp(year=date.year, month=quarter_start_month, day=1) 
    return str(first_day_of_quarter)[0:10]

In [154]:
first_day_of_quarter_func('2024-07-08')

'2024-07-01'

### 当年第一天

In [155]:
def first_day_of_year_func(date):
    date = pd.to_datetime(date)
    # 设置日期为该季度的第一天  
    first_day_of_year = pd.Timestamp(year=date.year, month=1, day=1) 
    return str(first_day_of_year)[0:10]

In [158]:
first_day_of_year_func('2024-10-23')

'2024-01-01'

### 聚合

In [167]:
dfi = dd[(dd.代理商id=='1505810869882216449')]
dfi = dfi.reset_index(drop=True)
dfi

,日期,代理商id,当日销售额
0,2021-01-01,1505810869882216449,388684.0
1,2021-01-02,1505810869882216449,235560.0
2,2021-01-03,1505810869882216449,338014.0
3,2021-01-04,1505810869882216449,203288.0
4,2021-01-05,1505810869882216449,134530.0
...,...,...,...
1189,2024-04-04,1505810869882216449,130838.0
1190,2024-04-05,1505810869882216449,203100.0
1191,2024-04-06,1505810869882216449,295154.0
1192,2024-04-07,1505810869882216449,87958.0


In [198]:
def func(ser):
    date = ser['日期']
    date_col = dfi['日期']
    
    当周累计销售额 = dfi[(date_col<=date) & (date_col>=first_day_of_week_func(date))]['当日销售额'].sum()
    
    当月累计销售额 = dfi[(date_col<=date) & (date_col>=first_day_of_month_func(date))]['当日销售额'].sum()
    
    当季累计销售额 = dfi[(date_col<=date) & (date_col>=first_day_of_quarter_func(date))]['当日销售额'].sum()
    
    当年累计销售额 = dfi[(date_col<=date) & (date_col>=first_day_of_year_func(date))]['当日销售额'].sum()
    
    历史累计销售额 = dfi[(date_col<=date)]['当日销售额'].sum()
    
    return pd.Series({
        
        ''
    
        '当周累计销售额': 当周累计销售额,
        '当月累计销售额': 当月累计销售额,
        '当季累计销售额': 当季累计销售额,
        '当年累计销售额': 当年累计销售额,
        '历史累计销售额': 历史累计销售额
        
    })
    
    
ri = dfi.apply(func, axis=1)
ri

,当周累计销售额,当月累计销售额,当季累计销售额,当年累计销售额,历史累计销售额
0,388684.0,388684.0,388684.0,388684.0,388684.0
1,624244.0,624244.0,624244.0,624244.0,624244.0
2,962258.0,962258.0,962258.0,962258.0,962258.0
3,203288.0,1165546.0,1165546.0,1165546.0,1165546.0
4,337818.0,1300076.0,1300076.0,1300076.0,1300076.0
...,...,...,...,...,...
1189,306550.0,306550.0,306550.0,13945260.0,312543471.0
1190,509650.0,509650.0,509650.0,14148360.0,312746571.0
1191,804804.0,804804.0,804804.0,14443514.0,313041725.0
1192,892762.0,892762.0,892762.0,14531472.0,313129683.0


In [201]:
agent_i = pd.concat([dfi, ri], axis=1)
agent_i

,日期,代理商id,当日销售额,当周累计销售额,当月累计销售额,当季累计销售额,当年累计销售额,历史累计销售额
0,2021-01-01,1505810869882216449,388684.0,388684.0,388684.0,388684.0,388684.0,388684.0
1,2021-01-02,1505810869882216449,235560.0,624244.0,624244.0,624244.0,624244.0,624244.0
2,2021-01-03,1505810869882216449,338014.0,962258.0,962258.0,962258.0,962258.0,962258.0
3,2021-01-04,1505810869882216449,203288.0,203288.0,1165546.0,1165546.0,1165546.0,1165546.0
4,2021-01-05,1505810869882216449,134530.0,337818.0,1300076.0,1300076.0,1300076.0,1300076.0
...,...,...,...,...,...,...,...,...
1189,2024-04-04,1505810869882216449,130838.0,306550.0,306550.0,306550.0,13945260.0,312543471.0
1190,2024-04-05,1505810869882216449,203100.0,509650.0,509650.0,509650.0,14148360.0,312746571.0
1191,2024-04-06,1505810869882216449,295154.0,804804.0,804804.0,804804.0,14443514.0,313041725.0
1192,2024-04-07,1505810869882216449,87958.0,892762.0,892762.0,892762.0,14531472.0,313129683.0


# 与历史日期对比

## 与N年前的数据进行对比
- 在时间维度上减去N年得到的日期，作为要对比的历史日期

In [207]:
agent_i

,日期,代理商id,当日销售额,当周累计销售额,当月累计销售额,当季累计销售额,当年累计销售额,历史累计销售额
0,2021-01-01,1505810869882216449,388684.0,388684.0,388684.0,388684.0,388684.0,388684.0
1,2021-01-02,1505810869882216449,235560.0,624244.0,624244.0,624244.0,624244.0,624244.0
2,2021-01-03,1505810869882216449,338014.0,962258.0,962258.0,962258.0,962258.0,962258.0
3,2021-01-04,1505810869882216449,203288.0,203288.0,1165546.0,1165546.0,1165546.0,1165546.0
4,2021-01-05,1505810869882216449,134530.0,337818.0,1300076.0,1300076.0,1300076.0,1300076.0
...,...,...,...,...,...,...,...,...
1189,2024-04-04,1505810869882216449,130838.0,306550.0,306550.0,306550.0,13945260.0,312543471.0
1190,2024-04-05,1505810869882216449,203100.0,509650.0,509650.0,509650.0,14148360.0,312746571.0
1191,2024-04-06,1505810869882216449,295154.0,804804.0,804804.0,804804.0,14443514.0,313041725.0
1192,2024-04-07,1505810869882216449,87958.0,892762.0,892762.0,892762.0,14531472.0,313129683.0


In [215]:
def 年同比(df, date_col, value_col, N=1):
    '''
    df: DataFrame对象
    date: 列名，指定以那一列作为日期,yyyy-mm-dd 字符串型
    value：值，指定以那一列作为对比的值
    N：默认位1，与前一年的数据作对比
    '''
    df1 = df.copy()
    
    df2 = df1[[date_col, value_col]]
    df2.columns = ['对比日期', value_col+'对比值']
    
    df1['日期时间型'] = pd.to_datetime(df1[date_col])
    df1['对比日期']   = (df1['日期时间型']-pd.DateOffset(years=N)).apply(lambda x:str(x)[0:10])
    df1 = df1.drop(columns=['日期时间型'])
    
    df3 = df1.merge(df2, how='left', on='对比日期')
    
    df3[value_col+'对比差异'] = df3[value_col]-df3[value_col+'对比值']
    df3[value_col+'差异百分比'] = df3[value_col+'对比差异'] / df3[value_col+'对比值']
    df3[value_col+'差异百分比'] = df3[value_col+'差异百分比'].replace(np.inf, np.nan)
    
    print('请确保创建了完整维度表!')
    
    return df3

In [216]:
年同比(agent_i, '日期', '当年累计销售额', 1)

请确保创建了完整维度表!


,日期,代理商id,当日销售额,当周累计销售额,当月累计销售额,当季累计销售额,当年累计销售额,历史累计销售额,对比日期,当年累计销售额对比值,当年累计销售额对比差异,当年累计销售额差异百分比
0,2021-01-01,1505810869882216449,388684.0,388684.0,388684.0,388684.0,388684.0,388684.0,2020-01-01,NaN,NaN,NaN
1,2021-01-02,1505810869882216449,235560.0,624244.0,624244.0,624244.0,624244.0,624244.0,2020-01-02,NaN,NaN,NaN
2,2021-01-03,1505810869882216449,338014.0,962258.0,962258.0,962258.0,962258.0,962258.0,2020-01-03,NaN,NaN,NaN
3,2021-01-04,1505810869882216449,203288.0,203288.0,1165546.0,1165546.0,1165546.0,1165546.0,2020-01-04,NaN,NaN,NaN
4,2021-01-05,1505810869882216449,134530.0,337818.0,1300076.0,1300076.0,1300076.0,1300076.0,2020-01-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1189,2024-04-04,1505810869882216449,130838.0,306550.0,306550.0,306550.0,13945260.0,312543471.0,2023-04-04,17829888.0,-3884628.0,-0.217872
1190,2024-04-05,1505810869882216449,203100.0,509650.0,509650.0,509650.0,14148360.0,312746571.0,2023-04-05,17964414.0,-3816054.0,-0.212423
1191,2024-04-06,1505810869882216449,295154.0,804804.0,804804.0,804804.0,14443514.0,313041725.0,2023-04-06,18047870.0,-3604356.0,-0.199711
1192,2024-04-07,1505810869882216449,87958.0,892762.0,892762.0,892762.0,14531472.0,313129683.0,2023-04-07,18117130.0,-3585658.0,-0.197915


In [220]:
agent_i[agent_i.日期=='2023-04-08']

,日期,代理商id,当日销售额,当周累计销售额,当月累计销售额,当季累计销售额,当年累计销售额,历史累计销售额
827,2023-04-08,1505810869882216449,132022.0,595466.0,1239328.0,1239328.0,18249152.0,242590917.0
